In [1]:
import pickle
from tqdm import tqdm_notebook
from joblib import Parallel, delayed
import requests

# для сбора лайков/дизлайков
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from youtube_data import youtube_search

# Это просто теория:

### Для работы подключаем YouTube Data API
* [пошаговая инструкция](https://medium.com/greyatom/youtube-data-in-python-6147160c5833)
* [документация](https://developers.google.com/youtube/v3/getting-started)

* [далее пользуемся инструкцией коллег из России](https://nbviewer.jupyter.org/github/DmitrySerg/top-russian-music/blob/master/parsers/clean_youtube_parser.ipynb)

* [и Индии](https://github.com/sudharsanasai/YoutubeDataAnalysis/blob/master/Youtube%20Data.ipynb)

In [2]:
with open('YT_api_key.txt') as f:
    api_key = f.read()

**Отправляем запрос серверу YouTube:**

In [3]:
base_search_url = 'https://www.googleapis.com/youtube/v3/search?'
url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, "UCs6h0i7chZZX3fWD4IOh1kQ")
req = requests.get(url)
req

<Response [200]>

Получаем `<Response [200]>` — [восторг!](https://developer.mozilla.org/ru/docs/Web/HTTP/Status/200)

### [Пробуем Индийские разработки:](https://github.com/sudharsanasai/YoutubeDataAnalysis/blob/master/Youtube%20Data.ipynb)

**сделаем тестовый запрос и сложим данные в таблицу `pandas`**

In [15]:
test = youtube_search("антоха мс")
test_df = pd.DataFrame(data=test)
test_df.head(2)

,tags,channelId,channelTitle,categoryId,title,videoId,viewCount,likeCount,dislikeCount,commentCount,favoriteCount
0,"[антоха мс, антохамс, antohamc, antoha mc, amc...",UCKLObxxmmAN4bBXdRtdqEJA,Антоха МС,10,Антоха МС - Коробка,MhidNQ41mwE,473820,49107,1365,1296,0
1,"[антохамс, антоха мс, максим томаш, томаш, tom...",UCKLObxxmmAN4bBXdRtdqEJA,Антоха МС,10,Антоха МС - Время Ток,JDSPAPOUU0U,8545710,173377,5928,4523,0


In [16]:
# запишем ценное:
test_df.to_csv('antoha_info')

### Все отработало здорово, пора исследовать. Берем код русских коллег:

In [4]:
def get_all_video_in_channel(channel_id, api_key):
    """
    Достаёт словарь из id и имен всех видео с канала по его id
        channel_id : string
            id канала
        api_key : string
            ключ доступа к API
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/search?'
    
    first_url = base_search_url+'key={}&channelId={}&part=snippet,id&order=date&maxResults=25'.format(api_key, channel_id)

    video_links = { }
    url = first_url
    while True:
        req = requests.get(url)
        resp = req.json()

        for i in resp['items']:
            if i['id']['kind'] == "youtube#video":
                video_links.update({i['id']['videoId'] : i['snippet']['title']})

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


def get_all_video_in_playlist(playlist_id, api_key):
    """
    Достаёт словарь из id и имен всех видео с плейлиста по его id
        playlist_id : string
            id канала
        api_key : string
            ключ доступа к API
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/playlistItems?'
    
    first_url = base_search_url+'key={}&playlistId={}&part=id,snippet&order=date&maxResults=25'.format(api_key, playlist_id)

    video_links = { }
    url = first_url
    while True:
        req = requests.get(url)
        resp = req.json()

        for i in resp['items']:
            if i['snippet']['resourceId']['kind'] == "youtube#video":
                video_links.update({i['snippet']['resourceId']['videoId'] : i['snippet']['title']})

        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break
    return video_links


def get_comment_threads(video_id, api_key, style, performer, video_dict, limit=2000):
    """
    Достёт из-под видео 500 комментов
        video_id : string 
            id видяшки
        api_key : string
            ключ доступа к API 
        style : string
            стиль музыки 
        performer : string
            исполнитель
        video_divt : dict
            словарь с соответствием id и названий видео
    """

    base_search_url = 'https://www.googleapis.com/youtube/v3/commentThreads?'
    first_url = base_search_url+'key={}&videoId={}&part=snippet&maxResults=100&textFormat=plainText'.format(api_key, video_id)

    com_inf = [ ]
    url = first_url

    while len(com_inf) < limit:
        req = requests.get(url)
        resp = req.json()

        for item in resp['items']:
            comment = item["snippet"]["topLevelComment"]
            if comment['kind'] == 'youtube#comment':
                cur_infa = {
                    'music_style' : style,
                    'performer' : performer,
                    'video_name' : video_dict[video_id],
                    'comment_id' : comment["id"],
                    'author' : comment["snippet"]["authorDisplayName"],
                    'text' : comment["snippet"]["textDisplay"],
                    'date' : comment["snippet"]['publishedAt'],
                    'likes' : comment["snippet"]['likeCount']
                }

                com_inf.append(cur_infa)       
        try:
            next_page_token = resp['nextPageToken']
            url = first_url + '&pageToken={}'.format(next_page_token)
        except:
            break    
    return com_inf

In [5]:
# делаем словарь, в который соберем комментарии
comments_dict = { }

### Получено:

In [ ]:

{
        'FACE' : {'канал': 'UCQpkqJ4bgocF5bwQsh-qHgg', 'листы': [
        'PL8PKfvXoO2-4NSEg5PBIfhpaHYh3HjPoy', 
        'PL8PKfvXoO2-7TfxrKbOUwjJ-n1cfkbNug', 
        'PL8PKfvXoO2-4CXqqeZPhGHRJ_pj6gWeUp'
    ]}
}


{
     'элджей' : {'канал': 'UCKm_XqwBamWfKBrwAZjmQGw', 'листы': [
        'PL79bk7Abmp0HzP2UdMLkhy_ZlKmiTQwzS', 
        'PL79bk7Abmp0H4z_qVFXLsL-T0-r3a-Hjp', 
        'PL79bk7Abmp0GBUgsOeh5RNH2fMTQRvzMj'
    ]}
 
    
{
     'ATL' : {'канал': 'UCxTuL5Rh8xDSD4IcYhthaaQ', 'листы': [
        'PLM0JG_nxoTpEPxauR-3B3FKCdh2-quiEo',
        'PLM0JG_nxoTpHjrL4L21u9Wl1quGv_9rQk', 
        'PLM0JG_nxoTpHWwTfBbKYsn-1BSef3gccK'
    ]}
    
#  качаю в 2 захаода, тк в первом скачано тольк 2700 комментов + 2700
{
    'ATL' : {'канал': 'UCxTuL5Rh8xDSD4IcYhthaaQ', 'листы': [
        'PLM0JG_nxoTpH1CKjJ7Z1YqvXQLf3y12mE',
        'PLM0JG_nxoTpH7gH_M4VWYi2-3ttmGH_uP',  
    
}

#         здесь хавтит одного плейлиста 66000 comments
{
     'oxxxymironofficial' : {'канал': 'UCCzp3DWpSc0s5wXYDghjM9A', 'листы': [
        'PLI80dHrUcwa0AeGn72hRzCPQGizU_8HbK'
    ]}
}

# 20 000        
{
     'antoha' : {'канал': 'UCKLObxxmmAN4bBXdRtdqEJA', 'листы': [
        'PL4nLGYL7NdIPtOEpZJlBaMcKI6aUfvPmZ'
    ]}
}
# еще Антоха
        
{
     'antoha' : {'канал': 'UCKLObxxmmAN4bBXdRtdqEJA', 'листы': [
        'PL4nLGYL7NdIPS1thwiDFIaJO09Nkbg8Fn'
    ]}
}     

## А этот не очереди:

In [19]:
style = 'rus-hip-hop'

target = {
#      'antoha' : {'канал': 'UCKLObxxmmAN4bBXdRtdqEJA', 'листы': [
#         'PL4nLGYL7NdIPS1thwiDFIaJO09Nkbg8Fn'
    ]}
}

In [20]:
for performer in target.keys():
    print('Качаю {}'.format(performer))
    
    channel_id = target[performer]['канал']
    playlist_ids =  target[performer]['листы']

    # видосы с основного канала 
    video_chanel = get_all_video_in_channel(channel_id, api_key)

    # видосы с плейлистов
    video_playlist = { }
    for lst in playlist_ids:
        video_playlist.update(get_all_video_in_playlist(lst, api_key))

    print('Видео с канала:',len(video_chanel))
    print('Видео с плейлиста:',len(video_playlist))
    video_playlist.update(video_chanel)
    print('Видео всего:',len(video_playlist))
    
    # пошли комменты
    def getCom(x):
        try:
            return get_comment_threads(x, api_key, style, performer, video_playlist)
        except:
            print('Error in https://www.youtube.com/watch?time_continue=1&v={}'.format(x))
            return [ ]

    n_jobs = -1 # параллелим на все ядра 
    result = Parallel(n_jobs=n_jobs)(delayed(getCom)(
        text) for text in tqdm_notebook(list(video_playlist.keys())))
    
    comments = [ ]
    for item in result:
        comments.extend(item)

    print('Добыто комментариев', len(comments))
    
    # воткнуть сюда сохранялку
    comments_dict[performer] = comments
    with open('antoha2.pickle', 'wb') as f:
        pickle.dump(comments_dict, f)
    
    print("==================================================")

Качаю antoha
Видео с канала: 347
Видео с плейлиста: 4
Видео всего: 351



Добыто комментариев 21301


**Складываю всё в переменную `eldjei_full_comments`**

In [53]:
eldjei_comments_YouTube = [ ]
for item in comments_dict:
    eldjei_comments_YouTube.extend(comments_dict[item])
  
len(eldjei_comments_YouTube)

73607

**Посмотрим на информаицю(она храниться в словаре) в комментарий номер 9000:**

In [30]:
eldjei_comments_YouTube[9000]

{'music_style': 'rus-hip-hop',
 'performer': 'элджей',
 'video_name': 'Don Diablo &amp; Элджей - UFO',
 'comment_id': 'UgwwUruYjnfMQtR2-ml4AaABAg',
 'author': 'HD Moment',
 'text': 'Лёха респект',
 'date': '2019-11-16T14:29:12.000Z',
 'likes': 0}

**Чтобы обратиться к тексту комментария используем конструкцию:**

In [36]:
eldjei_comments_YouTube[9000]['text']

'Лёха респект'

In [41]:
# количество лайков соответственно:
eldjei_comments_YouTube[9000]['likes']

0

**Чтобы посчитать сколько лайков поставили фанаты Элджея друг другу используем конструкцию цикла:**

In [42]:
# сначала обяв
total_likes = 0
for comment in eldjei_comments_YouTube:
    total_likes += comment['likes']

In [47]:
total_likes

458739

### Соберем данные в таблицу (без `comment_id`):

In [56]:
# обращаюсь к методам pandas

eldjei_comments_YouTube = pd.DataFrame(full_comments,
             
# выбираю столбцы  
             
             columns=['music_style', 'performer', 'video_name', 'author', 'text', 'date', 'likes'])
           
             

In [57]:
eldjei_comments_YouTube.head()

,music_style,performer,video_name,author,text,date,likes
0,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,The Best,Этим можно мазаться!,2019-12-16T20:25:24.000Z,0
1,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,The Best,"Здесь есть такие люди, которые реально фанаты ...",2019-12-16T20:22:35.000Z,1
2,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,The Best,Я надеюсь это будет в рекламе на ТВ!,2019-12-16T20:20:30.000Z,1
3,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,Динар Киньябузов,Не о чём,2019-12-16T15:10:29.000Z,0
4,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,Котайн топ,Тоооооооооппппп!!!!!!,2019-12-16T06:07:14.000Z,0


# ОТКРЫВАЕМ ФАЙЛЫ ДЛЯ РАБОТЫ:

In [43]:
# открываем наши файлы через pandas
eldjei_comments_YouTube = pd.read_pickle("eldjei_comments_YouTube.pickle")
FACE_comments_YouTube = pd.read_pickle("FACE_comments_YouTube.pickle")

eldjei_comments_YouTube

{'элджей': [{'music_style': 'rus-hip-hop',
   'performer': 'элджей',
   'video_name': 'SAYONARA BOY FW 19/20',
   'comment_id': 'UgxNzjButrLZAMtxvpx4AaABAg',
   'author': 'The Best',
   'text': 'Этим можно мазаться!',
   'date': '2019-12-16T20:25:24.000Z',
   'likes': 0},
  {'music_style': 'rus-hip-hop',
   'performer': 'элджей',
   'video_name': 'SAYONARA BOY FW 19/20',
   'comment_id': 'UgxVhIuKjmeAOc9wuU54AaABAg',
   'author': 'The Best',
   'text': 'Здесь есть такие люди, которые реально фанаты Элджея, и которым не раз говорили:" Элджей - ГОВНО!!!"?',
   'date': '2019-12-16T20:22:35.000Z',
   'likes': 1},
  {'music_style': 'rus-hip-hop',
   'performer': 'элджей',
   'video_name': 'SAYONARA BOY FW 19/20',
   'comment_id': 'Ugyc7mtkh69KPlSw7KF4AaABAg',
   'author': 'The Best',
   'text': 'Я надеюсь это будет в рекламе на ТВ!',
   'date': '2019-12-16T20:20:30.000Z',
   'likes': 1},
  {'music_style': 'rus-hip-hop',
   'performer': 'элджей',
   'video_name': 'SAYONARA BOY FW 19/20',
   

#### Повтаряем действия с формированием таблиц:

In [45]:
eldjei_data = [ ]
for item in eldjei_comments_YouTube:
    eldjei_data.extend(eldjei_comments_YouTube[item])
  
len(eldjei_data)

73607

In [46]:
eldjei_data = pd.DataFrame(eldjei_data,
             
# выбираю столбцы  
             
             columns=['music_style', 'performer', 'video_name', 'author', 'text', 'date', 'likes'])
           
eldjei_data.head()           

,music_style,performer,video_name,author,text,date,likes
0,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,The Best,Этим можно мазаться!,2019-12-16T20:25:24.000Z,0
1,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,The Best,"Здесь есть такие люди, которые реально фанаты ...",2019-12-16T20:22:35.000Z,1
2,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,The Best,Я надеюсь это будет в рекламе на ТВ!,2019-12-16T20:20:30.000Z,1
3,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,Динар Киньябузов,Не о чём,2019-12-16T15:10:29.000Z,0
4,rus-hip-hop,элджей,SAYONARA BOY FW 19/20,Котайн топ,Тоооооооооппппп!!!!!!,2019-12-16T06:07:14.000Z,0


## [Применим знания из лекции:](https://nbviewer.jupyter.org/github/FUlyankin/HSE_Data_Culture/blob/master/ML_for_marketing_2019/sems/sem1%262_data_prepare/sem1_sells.ipynb)

In [48]:
eldjei_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73607 entries, 0 to 73606
Data columns (total 7 columns):
music_style    73607 non-null object
performer      73607 non-null object
video_name     73607 non-null object
author         73607 non-null object
text           73607 non-null object
date           73607 non-null object
likes          73607 non-null int64
dtypes: int64(1), object(6)
memory usage: 3.9+ MB
